<a href="https://colab.research.google.com/github/SADHVIK1452/nlp/blob/main/Assignment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install Required Libraries

In [1]:
!pip install torch transformers


2. Import Libraries

In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random


3. Sample Text Data

In [3]:
# Sample text data (replace with a larger dataset if available)
text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""


4. Define Dataset Class for Tokenization

In [4]:
class TextDataset(Dataset):
    def __init__(self, text, tokenizer, max_length=128):
        self.examples = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=max_length)

    def __len__(self):
        return self.examples['input_ids'].size(0)

    def __getitem__(self, i):
        return {key: val[i] for key, val in self.examples.items()}


5. Initialize Model and Tokenizer

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

6. Prepare Dataset and DataLoader

In [6]:
# Set the padding token for the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Prepare Dataset and DataLoader
dataset = TextDataset(text, tokenizer)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)


7. Define Training Function

In [7]:
def train(model, data_loader, optimizer, scheduler, num_epochs=3):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in data_loader:
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['input_ids'])
            loss = outputs.loss
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

        print(f"Epoch {epoch + 1}/{num_epochs} Loss: {total_loss / len(data_loader)}")


8. Set Up Optimizer and Scheduler, Train the Model with Multiple Epochs

In [8]:
epochs_list = [20, 60, 70]  # Training for 20, 60, 70 epochs
for epochs in epochs_list:
    optimizer = AdamW(model.parameters(), lr=5e-5)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=epochs * len(data_loader))
    print(f"\nTraining for {epochs} epochs")
    train(model, data_loader, optimizer, scheduler, num_epochs=epochs)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Training for 20 epochs
Epoch 1/20 Loss: 6.395622730255127
Epoch 2/20 Loss: 3.558323621749878
Epoch 3/20 Loss: 2.1605799198150635
Epoch 4/20 Loss: 1.4830950498580933
Epoch 5/20 Loss: 1.2431060075759888
Epoch 6/20 Loss: 1.3036381006240845
Epoch 7/20 Loss: 1.105279564857483
Epoch 8/20 Loss: 0.9879280924797058
Epoch 9/20 Loss: 0.9213306307792664
Epoch 10/20 Loss: 0.8896371126174927
Epoch 11/20 Loss: 0.8776933550834656
Epoch 12/20 Loss: 0.7393894791603088
Epoch 13/20 Loss: 0.7482242584228516
Epoch 14/20 Loss: 0.7371389865875244
Epoch 15/20 Loss: 0.6592264175415039
Epoch 16/20 Loss: 0.6293419003486633
Epoch 17/20 Loss: 0.6255022287368774
Epoch 18/20 Loss: 0.5708460807800293
Epoch 19/20 Loss: 0.5681539177894592
Epoch 20/20 Loss: 0.559039294719696

Training for 60 epochs


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/60 Loss: 0.5316101312637329
Epoch 2/60 Loss: 0.46909910440444946
Epoch 3/60 Loss: 0.303232342004776
Epoch 4/60 Loss: 0.23319272696971893
Epoch 5/60 Loss: 0.2471667379140854
Epoch 6/60 Loss: 0.0955924242734909
Epoch 7/60 Loss: 0.07005172222852707
Epoch 8/60 Loss: 0.10091893374919891
Epoch 9/60 Loss: 0.06104085594415665
Epoch 10/60 Loss: 0.02830549143254757
Epoch 11/60 Loss: 0.01760949194431305
Epoch 12/60 Loss: 0.018302183598279953
Epoch 13/60 Loss: 0.012220814824104309
Epoch 14/60 Loss: 0.07845334708690643
Epoch 15/60 Loss: 0.009303074330091476
Epoch 16/60 Loss: 0.08621139079332352
Epoch 17/60 Loss: 0.008341516368091106
Epoch 18/60 Loss: 0.006320991087704897
Epoch 19/60 Loss: 0.015847697854042053
Epoch 20/60 Loss: 0.004827629309147596
Epoch 21/60 Loss: 0.005304012913256884
Epoch 22/60 Loss: 0.008482549339532852
Epoch 23/60 Loss: 0.007720746099948883
Epoch 24/60 Loss: 0.06659407913684845
Epoch 25/60 Loss: 0.003888525301590562
Epoch 26/60 Loss: 0.004544510971754789
Epoch 27/60 Lo

9. Define Text Generation Function

In [9]:
def generate_text(seed_text, max_length=50):
    model.eval()
    input_ids = tokenizer.encode(seed_text, return_tensors='pt')
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7, top_k=50)
    return tokenizer.decode(output[0], skip_special_tokens=True)


10. Generate Text with Initial Seed

In [10]:
seed_text = "Once upon a time"
generated_text = generate_text(seed_text)
print("Generated Text:\n", generated_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Text:
 Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her aunt. On her way through the


Experimenting and Improving the Model by large dataset and hyper tune parameter.

1. Try Different Learning Rates

In [11]:
learning_rates = [5e-5, 3e-5, 1e-4]  # Suggested learning rates to try

for lr in learning_rates:
    print(f"\nTraining with Learning Rate: {lr}")
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=20 * len(data_loader))  # For example, 20 epochs
    train(model, data_loader, optimizer, scheduler, num_epochs=20)  # Testing with 20 epochs for each learning rate



Training with Learning Rate: 5e-05
Epoch 1/20 Loss: 0.00029787796665914357
Epoch 2/20 Loss: 0.00020079406385775656
Epoch 3/20 Loss: 0.0001894389424705878
Epoch 4/20 Loss: 0.05919340252876282
Epoch 5/20 Loss: 0.00010684778681024909
Epoch 6/20 Loss: 0.00016097768093459308
Epoch 7/20 Loss: 0.0003454666875768453
Epoch 8/20 Loss: 0.05579507723450661
Epoch 9/20 Loss: 0.0001336520945187658
Epoch 10/20 Loss: 0.05775842443108559
Epoch 11/20 Loss: 9.762289118953049e-05
Epoch 12/20 Loss: 0.0008559277630411088
Epoch 13/20 Loss: 0.00022388815705198795
Epoch 14/20 Loss: 0.05826752260327339
Epoch 15/20 Loss: 0.0001314909604843706
Epoch 16/20 Loss: 0.0008755202288739383
Epoch 17/20 Loss: 6.712916365358979e-05
Epoch 18/20 Loss: 0.02904297225177288
Epoch 19/20 Loss: 7.067730621201918e-05
Epoch 20/20 Loss: 0.00012853331281803548

Training with Learning Rate: 3e-05
Epoch 1/20 Loss: 0.00033246082603000104
Epoch 2/20 Loss: 0.00010642891720635816
Epoch 3/20 Loss: 0.00028403272153809667
Epoch 4/20 Loss: 7.61

2. Experiment with Batch Size

In [12]:
batch_sizes = [1, 2, 4]  # Batch sizes to test

for batch_size in batch_sizes:
    print(f"\nTraining with Batch Size: {batch_size}")
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = AdamW(model.parameters(), lr=5e-5)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=20 * len(data_loader))  # For example, 20 epochs
    train(model, data_loader, optimizer, scheduler, num_epochs=20)  # Testing with 20 epochs for each batch size



Training with Batch Size: 1
Epoch 1/20 Loss: 5.100173075334169e-05
Epoch 2/20 Loss: 0.041630618274211884
Epoch 3/20 Loss: 0.004056244622915983
Epoch 4/20 Loss: 7.531924347858876e-05
Epoch 5/20 Loss: 6.199820199981332e-05
Epoch 6/20 Loss: 8.385186811210588e-05
Epoch 7/20 Loss: 6.092095281928778e-05
Epoch 8/20 Loss: 6.115033465903252e-05
Epoch 9/20 Loss: 5.466300353873521e-05
Epoch 10/20 Loss: 0.0448061041533947
Epoch 11/20 Loss: 5.94149278185796e-05
Epoch 12/20 Loss: 0.04397052153944969
Epoch 13/20 Loss: 7.78597459429875e-05
Epoch 14/20 Loss: 5.441291068564169e-05
Epoch 15/20 Loss: 6.820257112849504e-05
Epoch 16/20 Loss: 0.03904429450631142
Epoch 17/20 Loss: 6.496001151390374e-05
Epoch 18/20 Loss: 4.152030305704102e-05
Epoch 19/20 Loss: 5.396330016083084e-05
Epoch 20/20 Loss: 0.039574891328811646

Training with Batch Size: 2
Epoch 1/20 Loss: 5.2058509027119726e-05
Epoch 2/20 Loss: 5.0078895583283156e-05
Epoch 3/20 Loss: 4.130753586650826e-05
Epoch 4/20 Loss: 0.026107674464583397
Epoch 

3. Experiment with max_length in Text Generation

In [13]:
seed_text = "Once upon a time"
max_lengths = [50, 100, 150]  # Different max lengths for generation

for max_len in max_lengths:
    print(f"\nGenerating text with max_length: {max_len}")
    generated_text = generate_text(seed_text, max_length=max_len)
    print("Generated Text:\n", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generating text with max_length: 50


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
 Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandma. On her way through the

Generating text with max_length: 100


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
 Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandma. On her way through the forest, she met a big bad wolf who wanted to eat her.

Generating text with max_length: 150
Generated Text:
 Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandma. On her way through the forest, she met a big bad wolf who wanted to eat her.
